In [175]:
# Import the drive and mount the given drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [176]:
# import the Fictional dataset
import pandas as pd

data = pd.read_csv('Amazon_Reviews.txt', sep='\t')
data['Feedback'] = data['Feedback'].fillna('')
data.head(20)

,Product_Name,Customer_Rating,Review,Feedback
0,A,Product A is Good,5,
1,A,Product A is Good,5,
2,A,Product A is Good,5,
3,A,Product A is Good,5,
4,A,Product A is Good,5,
5,A,Product A is Good,5,
6,A,Product A is Good,5,
7,A,Product A is Good,5,
8,A,Product A is Good,5,
9,A,Product A is Good,5,


In [166]:
print(data.columns)

Index(['Product_Name', 'Customer_Rating ', 'Review', 'Feedback'], dtype='object')


In [167]:
data.columns = data.columns.str.strip()

In [170]:
mydata = data[['Customer_Rating','Feedback']]
mydata.columns = ['review','label']
mydata.head()

,review,label
0,Product A is Good,
1,Product A is Good,
2,Product A is Good,
3,Product A is Good,
4,Product A is Good,


In [171]:
mydata.value_counts('label')

,count
label,
,20


In [101]:
# Count the occurrences of each label
label_counts = mydata["label"].value_counts()

# Get the number of rows to drop from the majority class
rows_to_drop = label_counts.max() - label_counts.min()

# Drop rows from the majority class randomly
if rows_to_drop > 0:
   data_majority = mydata[mydata["label"] == 1]
   data_balanced = mydata.drop(data_majority.sample(rows_to_drop).index)
else:
   data_balanced = mydata.copy()

# Check the new class balance
print(data_balanced["label"].value_counts())

label
1    10
2    10
Name: count, dtype: int64


In [98]:
import re

def clean_text(text):
  # Remove special characters and punctuation
  text = re.sub(r"[^\w\s]", " ", text)

  # Remove single characters
  text = re.sub(r"\b[a-zA-Z]\b", " ", text)

  # Remove HTML tags
  text = re.sub(r"<[^>]*>", " ", text)

  # Lowercase the text
  text = text.lower()

  # Remove extra whitespace
  text = re.sub(r"\s+", " ", text)

  # Trim leading and trailing spaces
  text = text.strip()

  return text

In [99]:
import pandas as pd

# Extract the review column as a list
reviews = data_balanced['review'].tolist()

# Clean the text in the list, handling non-string values
cleaned_reviews = [clean_text(review) if isinstance(review, str) else '' for review in reviews]

# Add the cleaned reviews as a new column to the DataFrame
data_balanced['clean_reviews'] = cleaned_reviews

In [105]:
import pandas as pd

# Assuming your DataFrame is called "df"
total_rows = len(data_balanced)
test_size = int(total_rows * 0.95)

# Randomly sample train_size rows for the training set
test_set = data_balanced.sample(test_size)

# Get the remaining rows for the test set
train_set = data_balanced.drop(test_set.index)

In [111]:
import pandas as pd

# Assuming your DataFrame is called "df"
total_rows = len(data_balanced)
test_size = int(total_rows * 1)

# Randomly sample train_size rows for the training set
test_set = data_balanced.sample(test_size)

# Get the remaining rows for the test set
train_set = data_balanced.drop(test_set.index)

In [90]:
!pip install openai==0.27.0

In [91]:
import openai
from google.colab import userdata

OPENAI_API_KEY=userdata.get('OPENAI_API_KEY')
openai.api_key  = OPENAI_API_KEY

In [92]:
def get_completion(prompt, model="gpt-3.5-turbo-1106"):

  messages = [{"role": "user", "content": prompt}]
  response = openai.ChatCompletion.create(model=model,messages=messages,temperature=0)

  return response.choices[0].message["content"]

In [146]:
prompt = "Test Prompt : tell me about Boston"


In [172]:
chatgpt_response

"Boston is the capital and largest city of the state of Massachusetts in the United States. It is one of the oldest cities in the country and has a rich history dating back to the colonial era. \n\nBoston is known for its important role in the American Revolution, with iconic sites such as the Boston Tea Party Ships & Museum and the Freedom Trail, which takes visitors on a journey through the city's revolutionary past. \n\nThe city is also home to several prestigious universities, including Harvard University and the Massachusetts Institute of Technology, making it a hub for education and innovation.\n\nBoston is famous for its sports teams, including the Red Sox (baseball), Celtics (basketball), and Bruins (hockey), and the city has a passionate sports culture.\n\nThe city is also known for its vibrant arts and culture scene, with numerous museums, theaters, and music venues. The Boston Symphony Orchestra and the Boston Pops are renowned for their performances.\n\nBoston is a bustling

In [108]:
test_set.shape

(19, 2)

In [118]:
test_set_total = test_set.sample(20)

test_set_total['pred_label'] = ''

test_set_total

,review,label,pred_label
10,Product B is Bad,2,
11,Product B is Bad,2,
0,Product A is Good,1,
2,Product A is Good,1,
7,Product A is Good,1,
12,Product B is Bad,2,
16,Product B is Bad,2,
8,Product A is Good,1,
9,Product A is Good,1,
4,Product A is Good,1,


In [148]:
import time

def gpt_completion_function(data, model="gpt-3.5-turbo-1106"):
  """Function works in three steps:
  # Step-1: Convert the DataFrame to JSON using the to_json() method.
  # Step-2: Preparing the Gemini Prompt
  # Step-3: Calling GPT API
  """
  json_data = mydata[['clean_reviews']].to_json(orient='records')

  prompt = f"""You are an expert linguist, who is good at classifying customer review sentiments into Positive/Negative labels.
  Help me classify customer reviews into: Positive(label=1), and Negative(label=0).
  Customer reviews are provided between three backticks below.
  In your output, only return the Json code back as output - which is provided between three backticks.
  Your task is to update predicted labels under 'pred_label' in the Json code.
  Don't make any changes to Json code format, please.
  Error handling instruction: In case a Customer Review violates API policy, please assign it default sentiment as Negative (label=0).

  ```
  {json_data}
  ```
  """

  print(prompt)

  messages = [{"role": "user", "content": prompt}]
  response = openai.ChatCompletion.create(model=model,messages=messages,temperature=0)
  time.sleep(5)
  return response.choices[0].message["content"]

# Calling the function with the entire dataset
response = gpt_completion_function(test_set_total)

# Process the response to create a DataFrame
json_data = response.strip("`")
data = json.loads(json_data)
df_total = pd.DataFrame(data)

print(df_total)  # Display the final DataFrame

You are an expert linguist, who is good at classifying customer review sentiments into Positive/Negative labels.
  Help me classify customer reviews into: Positive(label=1), and Negative(label=0).
  Customer reviews are provided between three backticks below.
  In your output, only return the Json code back as output - which is provided between three backticks.
  Your task is to update predicted labels under 'pred_label' in the Json code.
  Don't make any changes to Json code format, please.
  Error handling instruction: In case a Customer Review violates API policy, please assign it default sentiment as Negative (label=0).

  ```
  [{"clean_reviews":"product is good"},{"clean_reviews":"product is good"},{"clean_reviews":"product is good"},{"clean_reviews":"product is good"},{"clean_reviews":"product is good"},{"clean_reviews":"product is good"},{"clean_reviews":"product is good"},{"clean_reviews":"product is good"},{"clean_reviews":"product is good"},{"clean_reviews":"product is good"

In [149]:
# prompt: Overwrite pred_label from 'df' into pred_label in 'train_set_sample'

test_set_total['pred_label'] = df_total['pred_label'].values
test_set_total

,review,label,pred_label
10,Product B is Bad,2,1
11,Product B is Bad,2,1
0,Product A is Good,1,1
2,Product A is Good,1,1
7,Product A is Good,1,1
12,Product B is Bad,2,1
16,Product B is Bad,2,1
8,Product A is Good,1,1
9,Product A is Good,1,1
4,Product A is Good,1,1


Optional : Batch Processing Code

In [113]:
batches = []
batch_size = 20

for i in range(0, len(test_set_total), batch_size):
  batches.append(test_set_total[i : i + batch_size])  # Append batches instead of assigning

In [114]:
import time

def gpt_completion_function(batch,current_batch,total_batch,model="gpt-3.5-turbo-1106"):
  """Function works in three steps:
  # Step-1: Convert the DataFrame to JSON using the to_json() method.
  # Step-2: Preparing the Gemini Prompt
  # Step-3: Calling GPT API
  """

  print(f"Now processing batch#: {current_batch+1} of {total_batch}")

  json_data = batch[['clean_reviews','pred_label']].to_json(orient='records')

  prompt = f"""You are an expert linguist, who is good at classifying customer review sentiments into Positive/Negative labels.
  Help me classify customer reviews into: Positive(label=1), and Negative(label=0).
  Customer reviews are provided between three backticks below.
  In your output, only return the Json code back as output - which is provided between three backticks.
  Your task is to update predicted labels under 'pred_label' in the Json code.
  Don't make any changes to Json code format, please.
  Error handling instruction: In case a Customer Review violates API policy, please assign it default sentiment as Negative (label=0).

  ```
  {json_data}
  ```
  """

  print(prompt)

  messages = [{"role": "user", "content": prompt}]
  response = openai.ChatCompletion.create(model=model,messages=messages,temperature=0)
  time.sleep(5)
  return response.choices[0].message["content"]

In [37]:
batch_count = len(batches)
responses = []

for i in range(0,len(batches)):
  responses.append(gpt_completion_function(batches[i],i,batch_count))

Now processing batch#: 1 of 1
You are an expert linguist, who is good at classifying customer review sentiments into Positive/Negative labels.
  Help me classify customer reviews into: Positive(label=1), and Negative(label=0).
  Customer reviews are provided between three backticks below.
  In your output, only return the Json code back as output - which is provided between three backticks.
  Your task is to update predicted labels under 'pred_label' in the Json code.
  Don't make any changes to Json code format, please.
  Error handling instruction: In case a Customer Review violates API policy, please assign it default sentiment as Negative (label=0).

  ```
  [{"clean_reviews":"product   is bad","pred_label":""},{"clean_reviews":"product   is good","pred_label":""},{"clean_reviews":"product   is bad","pred_label":""},{"clean_reviews":"product   is good","pred_label":""},{"clean_reviews":"product   is bad","pred_label":""},{"clean_reviews":"product   is bad","pred_label":""},{"clean_

In [38]:
import pandas as pd

# Initialize df_total as an empty DataFrame if not already initialized
df_total = pd.DataFrame()

# Example of creating df_temp (replace this with your actual code)
df_temp = pd.DataFrame({'Column1': [1, 2], 'Column2': ['A', 'B']})  # Sample data

# Use pd.concat to combine DataFrames, ensuring df_temp exists
df_total = pd.concat([df_total, df_temp], ignore_index=True)

print(df_total)  # Display the final DataFrame

   Column1 Column2
0        1       A
1        2       B


In [39]:
import pandas as pd

# Assuming df_temp is your temporary DataFrame and df_total is the final DataFrame
# Initialize df_total as an empty DataFrame if not already initialized
df_total = pd.DataFrame()

# ... (your existing code to create df_temp)

# Use pd.concat to combine DataFrames
df_total = pd.concat([df_total, df_temp], ignore_index=True)

print(df_total)  # Display the final DataFrame


   Column1 Column2
0        1       A
1        2       B


In [115]:
import json
import pandas as pd

df_total = pd.DataFrame()  # Initialize an empty DataFrame

for response in responses:
    # Clean the data by stripping the backticks
    json_data = response.strip("`")

    # Load the cleaned data and convert to DataFrame
    data = json.loads(json_data)
    df_temp = pd.DataFrame(data)

    # Use pd.concat to combine DataFrames
    df_total = pd.concat([df_total, df_temp], ignore_index=True)

print(df_total)  # Display the final DataFrame

        clean_reviews  pred_label
0    product   is bad           0
1   product   is good           1
2    product   is bad           0
3   product   is good           1
4    product   is bad           0
5    product   is bad           0
6    product   is bad           0
7    product   is bad           0
8   product   is good           1
9    product   is bad           0
10   product   is bad           0
11  product   is good           1
12  product   is good           1
13  product   is good           1
14  product   is good           1
15  product   is good           1
16   product   is bad           0
17  product   is good           1
18  product   is good           1


In [41]:
# prompt: Overwrite pred_label from 'df' into pred_label in 'train_set_sample'

test_set_total['pred_label'] = df_total['pred_label'].values
test_set_total

,review,label,clean_reviews,pred_label
15,Product B is Bad,2,product is bad,0
4,Product A is Good,1,product is good,1
19,Product B is Bad,2,product is bad,0
1,Product A is Good,1,product is good,1
10,Product B is Bad,2,product is bad,0
12,Product B is Bad,2,product is bad,0
11,Product B is Bad,2,product is bad,0
13,Product B is Bad,2,product is bad,0
9,Product A is Good,1,product is good,1
18,Product B is Bad,2,product is bad,0


In [140]:
# Check the new class balance
print(test_set_total["pred_label"].value_counts())

pred_label
    20
Name: count, dtype: int64
